In [14]:
class Node:
    def __init__(self, key):
        self.key = key
        self.left = self.right = None
        
    def display(self):
        lines, _, _, _ = self._display_aux()
        for line in lines:
            print(line)


    ### Display from https://stackoverflow.com/questions/34012886/print-binary-tree-level-by-level-in-python/34014370
    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.key
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.key
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2



In [21]:
class Number(Node):
    def __init__(self, num):
        self.key = num
        self.right = self.left = None

class Operator(Node):
    def __init__(self, l, o, r):
        self.left = l
        self.right = r
        self.key = o


# Tupel to define the priority of the operators
priority = {'*': 3,
            '/': 3,     # punkt vor strich
            '+': 2,
            '-': 2,
            '(': 9,
            ')': 0}     # last


# Shunting-Yard Algorithm
def shunting_yard(infix):
    # If param is null raise exception
    if infix is None:
        raise SyntaxError

    # make a list from
    expression = list(infix)

    output, operatorStack = [], []

    for token in expression:
        # check if element is a digit
        if token.isdigit():
            output.append(token)
        # if not digit check if present in
        elif token in priority:
            while operatorStack:
                stackTop = operatorStack[-1]
                if priority[token] <= priority[stackTop]:
                    if token != ')':
                        if stackTop != '(':
                            operatorStack.pop()
                            output.append(stackTop)
                        else:
                            break
                    else:
                        if stackTop != '(':
                            operatorStack.pop()
                            output.append(stackTop)
                        else:
                            operatorStack.pop()
                            break
                else:
                    break

            if token != ')':
                operatorStack.append(token)

    while operatorStack:
        remainingItem = operatorStack[-1]
        operatorStack.pop()
        output.append(remainingItem)
    return output

def parse (input):
    shuntingYard = shunting_yard(input)

    if shuntingYard is None:
        raise SyntaxError

    stack = []

    for token in shuntingYard:
        if token in priority:
            stack.append(Operator(stack.pop(), token, stack.pop()))
        else:
            stack.append(Number(token))

    if len(stack) != 1:
        raise SyntaxError

    return stack[0]

## todo FIX THIS
def evaluateTree(root):
    if root is None:
        raise SyntaxError("tree is empty")

    # check if root is a instance of Number
    if isinstance(root, Number):
        return root.value

    leftEvaluation = int(evaluateTree(root.left))       # round to int if float
    rightEvaluation = int(evaluateTree(root.right))

    # evaluate operators
    if root.operator == '+':
        return rightEvaluation + leftEvaluation
    elif root.operator == '-':
        return rightEvaluation - leftEvaluation
    elif root.operator == '*':
        return rightEvaluation * leftEvaluation
    elif root.operator == '/':
        return rightEvaluation / leftEvaluation

In [28]:
class Solver:
    def evaluate(self, node):
        # Get either evaluate_Operator or evaluate_Number
        method_name = 'evaluate_' + type(node).__name__
        evaluator = getattr(self, method_name, self.generic_evaluate)
        return evaluator(node)
    
    def generic_evaluate(self, node):
        raise Exception(f"No evaluate_{type(node).__name__} method found!")
    
    def __init__(self, root):
        self.root = root
        
    def evaluate_Operator(self, node):
        if node.key == "+": 
            return self.evaluate(node.left) + self.evaluate(node.right)
        
        if node.key == "-": 
            return self.evaluate(node.left) - self.evaluate(node.right)

        if node.key == "*": 
            return self.evaluate(node.left) * self.evaluate(node.right)

        if node.key == ":" or node.key == "/": 
            return self.evaluate(node.left) / self.evaluate(node.right)

    def evaluate_Number(self, node):
        return int(node.key)
    
    def solve(self):
        return self.evaluate(self.root)



In [34]:
infix = '2*(5+3)*(4+7)'
print("Converted String polish form: {}", shunting_yard(infix)) # expect 2 4 * 3 4 7 − 8 * + * 1 6 − −  YAYY works right
converted= shunting_yard(infix)

# 2*(5+3)
solver = Solver(parse(converted))

solver.root.display()

print(solver.solve())

Converted String polish form: {} ['2', '5', '3', '+', '*', '4', '7', '+', '*']
      _+ 
     /  \
    _*  2
   /  \  
  _+  5  
 /  \    
 *  3    
/ \      
7 4      
157


In [33]:
infix = '2*4*(3+(4-7)*8)-(1-6)'
print("Converted String polish form: {}", shunting_yard(infix)) # expect 2 4 * 3 4 7 − 8 * + * 1 6 − −  YAYY works right
converted= shunting_yard(infix)

# 2*4*(3+(4-7)*8)-(1-6) = -163
solver = Solver(parse(converted))

solver.root.display()

print(solver.solve())

Converted String polish form: {} ['2', '4', '*', '3', '4', '7', '-', '8', '*', '+', '*', '1', '6', '-', '-']
            _- 
           /  \
    _______-  2
   /        \  
  _+_____   4  
 /       \     
 *    ___-     
/ \  /    \    
6 1  *_   3    
    /  \       
    8  *       
      / \      
      7 4      
221
